In [1]:
from google.colab import auth
auth.authenticate_user()

# Install Google Cloud Storage client
# !pip install --upgrade google-cloud-storage

from google.cloud import storage, bigquery

# Set your bucket name
BUCKET_NAME = 'wpac-bucket-1'

# Create a storage client
client = storage.Client()
bucket = client.bucket(BUCKET_NAME)

In [2]:
blobs = bucket.list_blobs()
for blob in blobs:
    print(blob.name)

WAP_site_resources.csv
WPA_all_time_connect.csv
WPA_pulse_survey.csv
WPA_site_jobs.csv
cleaned_jobs.csv


In [3]:
FILE_NAME = 'WAP_site_resources.csv'  # Change to your file path in GCS
blob = bucket.blob(FILE_NAME)
blob.download_to_filename('local-file.csv')  # Save as a local file
print(f'Downloaded {FILE_NAME} to local-file.csv')


Downloaded WAP_site_resources.csv to local-file.csv


In [4]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [5]:
resources = pd.read_csv('local-file.csv')
resources.head()

,data_careerId,featured,providerName,updatedAt,resource_tag
0,63e53d0f3f747a7a197b9808,Don't have a working computer at home? Would y...,Compudopt Dallas,2024-11-22T03:05:40Z,supplies
1,63e53d0f3f747a7a197b9808,Don't have a working computer at home? Would y...,Compudopt Dallas,2024-11-22T03:05:40Z,life resources
2,63e53d123f747a7a197b980e,Dallas City Hall can help you with social serv...,Dallas City Hall,2024-11-22T03:07:09Z,life resources
3,63e53d123f747a7a197b980e,Dallas City Hall can help you with social serv...,Dallas City Hall,2024-11-22T03:07:09Z,legal
4,63e53d123f747a7a197b980e,Dallas City Hall can help you with social serv...,Dallas City Hall,2024-11-22T03:07:09Z,housing


In [6]:
print(resources.nunique())
print(resources.info())

data_careerId    150
featured         150
providerName     135
updatedAt        143
resource_tag      26
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 737 entries, 0 to 736
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   data_careerId  737 non-null    object
 1   featured       737 non-null    object
 2   providerName   737 non-null    object
 3   updatedAt      737 non-null    object
 4   resource_tag   734 non-null    object
dtypes: object(5)
memory usage: 28.9+ KB
None


In [7]:
null_resource_tags = resources[resources['resource_tag'].isnull() == True]
for i in null_resource_tags['featured']:
  text = i.split()
  print(text)
# print(null_resource_tags['featured'])

['In', 'this', 'video,', 'we', 'chatted', 'with', 'Ron,', 'a', 'New', 'York', 'City', 'resident', 'and', 'SkillUp', 'advocate,', 'who', 'gained', 'career', 'confidence', 'through', 'networking', 'skills', 'learned', 'in', 'a', 'SkillUp/Social', 'Capital', 'Builders', 'workshop']
['Facing', 'the', 'need', 'for', 'diversified', 'skills', 'in', 'the', 'competitive', 'remote', 'work', 'landscape,', 'Jacoby', 'struggled', 'with', 'challenges', 'in', 'his', 'personal', 'identity', 'affecting', 'his', 'professional', 'history.', 'Learn', 'how', 'SkillUp', 'helped!']
['Learn', 'how', 'Stephanie', 'leveraged', 'SkillUp', 'to', 'find', 'career', 'training', 'and', 'support!']


In [8]:
resources['resource_tag'].value_counts(dropna=False)

,count
resource_tag,
life resources,104
career support,61
housing,56
education,51
food,49
job search,46
medical,45
financial,44
employment services,37


In [9]:
resources['resource_tag'] = resources['resource_tag'].fillna('worker stories')
print(resources['resource_tag'].isnull().sum())

0


In [10]:
resources.drop(columns=['featured', 'providerName', 'updatedAt'], axis=1, inplace=True)

print(resources.head())

              data_careerId    resource_tag
0  63e53d0f3f747a7a197b9808        supplies
1  63e53d0f3f747a7a197b9808  life resources
2  63e53d123f747a7a197b980e  life resources
3  63e53d123f747a7a197b980e           legal
4  63e53d123f747a7a197b980e         housing


In [11]:
# !pip install google-cloud-bigquery

In [16]:
project_id = "wpac-449904"
client = bigquery.Client(project=project_id)

In [13]:
resources.to_csv('cleaned_resources.csv', index=False)

from google.colab import files
# files.download('cleaned_resources.csv')

In [17]:
dataset_id = "wpac_data"

client = bigquery.Client(project=project_id)
datasets = [dataset.dataset_id for dataset in client.list_datasets()]
print("Available Datasets:", datasets)

if dataset_id not in datasets:
    print(f"Dataset {dataset_id} does not exist. Creating it now...")
    dataset_ref = client.dataset(dataset_id)
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = "US"  # Change if necessary
    client.create_dataset(dataset, exists_ok=True)
    print(f"Dataset {dataset_id} created.")


Available Datasets: ['wpac_data']


In [18]:
import pandas_gbq

dataset_table = "wpac_data.cleaned_resources"  # Replace with dataset.table name

# Load your CSV into a DataFrame
csv_file_path = "cleaned_resources.csv"  # Replace with your actual file path
# df = pd.read_csv(csv_file_path)

# Use `pandas_gbq.to_gbq` to upload DataFrame
pandas_gbq.to_gbq(
    resources,
    destination_table=dataset_table,
    project_id=project_id,
    if_exists="replace"  # "fail", "replace", "append"
)


100%|██████████| 1/1 [00:00<00:00, 6423.13it/s]
